#Replicacion del paper "Attention is all you need"

En este notebook se desarrolla la replicacion del paper "Attention is all you need" donde se detalla por primera vez la arquitectura completa de un modelo transformer encoder-decoder para la traduccion de texto. En este caso vamos a enfocarnos en la traduccion de texto del ingles al frances. Cabe aclarar que por limitaciones computacionales no se va a tomar el dataset completo sino una version mas acotada. Sin embargo, se va a intentar en medida de lo posible mantener la arquitectura original del paper. Esta arquitectura va a ser desarrollada desde cero con Tensorflow. Pasos generales del proyecto:
* Definir la arquitectura
* Descargar y preparar el dataset
* Entrenar el modelo
* Prueba con fine-tuning de LLM
* Hacer pruebas en la traduccion

## Definir la arquitectura

In [ ]:
!pip install tensorflow

In [ ]:
#Librerias de la seccion
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Embedding, LayerNormalization, Dropout, TextVectorization
import numpy as np
import os

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
print('Numero de dispositivos:', format(mirrored_strategy.num_replicas_in_sync))

Numero de dispositivos: 1


Entrenamiento con precision mixta. Entrenamos el modelo con una combinacion de datos de 16 y 32 bits para reducir el tiempo de entrenamiento.

In [ ]:
# Corrected way to set mixed precision policy
tf.keras.mixed_precision.set_global_policy('mixed_float16')

### Positional Encoding.

In [ ]:
# Positional Encoding
def pos_ratio(pos, i, d_model):
    pos_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    #print('Pos', pos)
    return pos * pos_rates

@tf.function
def positional_encoding(max_seq_length, d_model):

    #print('Position', max_seq_length)
    pos_ratios = pos_ratio(
        np.arange(max_seq_length)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model)

    pos_ratios[:, 0::2] = np.sin(pos_ratios[:, 0::2])
    pos_ratios[:, 1::2] = np.cos(pos_ratios[:, 1::2])
    pos_encoding = pos_ratios[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

### TextVectorization, Embedding + Positional Encoding
Combinamos el positional encoding con la vectorizacion de texto y el embedding en una capa a la que llamamos PreLayer que prepara los datos para ser transferidos al modelo.

enc_dec:
* 0 si es encoder
* 1 si es decoder

In [ ]:
class PreLayer(Layer):
  def __init__(self, d_model, vocab_size, max_seq_length, enc_dec = 0):
      super(PreLayer, self).__init__()
      self.d_model = d_model
      self.max_seq_length = max_seq_length
      self.enc_dec = enc_dec

      #Vectorizers
      #Para adapt y obtener vocabulario.
      if enc_dec == 0:
        self.vectorizer = TextVectorization(max_tokens= vocab_size, output_mode='int', output_sequence_length = max_seq_length, standardize = None)
      elif enc_dec == 1:
        self.vectorizer = TextVectorization(max_tokens= vocab_size, output_mode='int', output_sequence_length = max_seq_length + 1, standardize = None)

      #Embedding
      self.embedding = Embedding(input_dim= vocab_size, output_dim= d_model, mask_zero=True) #Mask Zero sirve para manejar los tokens de padding sin que se les asigne un peso significativo.

      #Positional Encoding
      self.pos_encoding = positional_encoding(max_seq_length, d_model)

      # Dropout
      self.dropout = Dropout(0.1)

  @tf.function
  def adapt(self, dataset):
    self.vectorizer.adapt(dataset)
    return self.vectorizer

  @tf.function
  def get_vocabulary(self):
    return self.vectorizer.get_vocabulary()

  @tf.function
  def call(self, inputs, training):

        x = inputs

        #print('Inputs',inputs)

        # Mascara de padding extra para la logica de atencion.
        mask = tf.cast(tf.not_equal(x, 0), tf.float32)

        #print('Padding Mask',mask)

        # Embedding y positional encoding
        x = self.embedding(x)
        #print('Embedded',x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32)) #Equilibra la magnitud del embedding y del positional encoding para que el modelo aprenda más rápido.
        seq_len = tf.shape(x)[1]
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)

        # Mascara para atenttion.
        padding_mask = mask[:, tf.newaxis, tf.newaxis, :]
        #print('Masked',padding_mask)
        return x, padding_mask

### Multi-Head Attention

In [ ]:
# Masking
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (max_seq_length, max_seq_length)

# Multi-head Attention
class MultiHeadAttention(Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // num_heads

        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)
        self.dense = Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0,2,1,3])  # (batch, h, max_seq_length, depth)

    @tf.function
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
        q = self.wq(q) # Pasamos x por la red densa para crear Q. (max_seq_length, d_model) x (d_model, d_model) = (max_seq_length, d_model)
        k = self.wk(k) # Pasamos x por la red densa para crear K. (max_seq_length, d_model) x (d_model, d_model) = (max_seq_length, d_model)
        v = self.wv(v) # Pasamos x por la red densa para crear V. (max_seq_length, d_model) x (d_model, d_model) = (max_seq_length, d_model)

        #Dividimos las dimensiones para pasarlas por las cabezas de atencion correspondientes.
        q = self.split_heads(q, batch_size) # (h, max_seq_length, depth)
        k = self.split_heads(k, batch_size) # (h, max_seq_length, depth)
        v = self.split_heads(v, batch_size) # (h, max_seq_length, depth)

        matmul_qk = tf.matmul(q, k, transpose_b=True) #Multiplicamos las matrices QxK con K traspuesta para matchear las shapes.
        dk = tf.cast(tf.shape(k)[-1], tf.float32) # Escalamos el resultado.
        pre_score = matmul_qk / tf.math.sqrt(dk) # Computamos el score.

        if mask is not None:
            pre_score += (mask * -1e9)

        attention_scores = tf.nn.softmax(pre_score, axis=-1) # Aplicamos softmax al score
        output = tf.matmul(attention_scores, v) # Multiplicamos los scores con la matriz V.

        output = tf.transpose(output, perm=[0,2,1,3])
        concat_attention = tf.reshape(output, (batch_size, -1, self.d_model)) # Concatenamos los resultados de cada head.
        out = self.dense(concat_attention) # Pasamos los resultados por una red densa de dimension d_model.
        return out # Salida de tamaño (max_seq_length, d_model)

### Encoder

In [ ]:
# Encoder Layer
class EncoderLayer(Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    @tf.function
    def call(self, x, training, mask):
        attn_output = self.mha(x, x, x, mask) # Computamos el MHA con la salida del PreLayer.
        attn_output = self.dropout1(attn_output, training=training) # Aplicamos Dropout para evitar overfitting.
        out1 = self.layernorm1(x + attn_output) # Hacemos el skip connection o conexion residual para no perder el contexto.

        ffn_output = self.ffn(out1) # Pasamos la salida del skip connection por el FFN. (max_seq_length, d_model) x (d_model, dff) => (max_seq_length, dff) x (dff, d_model) => (max_seq_length, d_model)
        ffn_output = self.dropout2(ffn_output, training=training) # Dropout de vuelta para reducir overfitting.
        out2 = self.layernorm2(out1 + ffn_output) # Hacemos nuevamente el skip connection para no perder este contexto.
        return out2

# Encoder
class Encoder(Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, max_seq_length, rate=0.1):
        super(Encoder, self).__init__()
        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                           for _ in range(num_layers)] # Creamos tantos encoders como le digamos.
        self.num_layers = num_layers

    @tf.function
    def call(self, inputs, training, mask):
      x = inputs
      for i in range(self.num_layers):
          x = self.enc_layers[i](x, training = training, mask = mask) # Pasamos la mascara de padding y el
      return x, mask  # (batch, max_seq_length, d_model)

### Decoder

In [ ]:
# Decoder Layer
class DecoderLayer(Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.layernorm3 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
        self.dropout3 = Dropout(rate)

    @tf.function
    def call(self, x, enc_output, training,
             look_ahead_mask, padding_mask):
        attn1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2 = self.mha2(enc_output, enc_output, out1, padding_mask) # En el segundo MHA vamos a pasar por Q y K la salida de los encoders.
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)
        return out3

# Decoder
class Decoder(Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 target_vocab_size, max_seq_length, rate=0.1):
        super(Decoder, self).__init__()

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                           for _ in range(num_layers)]
    @tf.function
    def call(self, inputs, enc_output, training,
             look_ahead_mask, padding_mask):
      x = inputs

      for i in range(len(self.dec_layers)):
          x = self.dec_layers[i](
              x, enc_output,
              training=training,
              look_ahead_mask=look_ahead_mask,
              padding_mask=padding_mask)
      return x  # (batch, max_seq_length, d_model)

### Transformer Encoder-Decoder Completo

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        #PreLayers
        self.pre_layer_encoder = PreLayer(d_model= d_model, vocab_size= input_vocab_size, max_seq_length=max_seq_length, enc_dec = 0)
        self.pre_layer_decoder = PreLayer(d_model= d_model, vocab_size= target_vocab_size, max_seq_length=max_seq_length, enc_dec= 1)

        #Encoder
        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

        #Decoder
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

        #Final Layer
        self.final_layer = Dense(target_vocab_size, activation='softmax') # Usamos el target vocab size para tener la cantidad de tokens como clases.

    @tf.function
    def call(self, inputs, training):
        encoder_inputs, decoder_inputs = inputs

        x_encoder, encoder_padding_mask = self.pre_layer_encoder(encoder_inputs, training = training) # Computamos Embedding y Positional Enconding en el PreLayer del Encoder

        enc_output, enc_mask = self.encoder(x_encoder, training=training, mask=encoder_padding_mask) # Pasamos la salida de la PreLayer del Encoder al Encoder.

        x_decoder,_ = self.pre_layer_decoder(decoder_inputs, training = training)

        seq_len = tf.shape(decoder_inputs)[1] # Obtenemos la longitud de las secuencias de los inputs del decoder.
        look_ahead_mask = create_look_ahead_mask(seq_len) # Creamos la mascara de look ahead.

        # Combinamos la look ahead mask con la de decoder
        # Máscara de padding para la secuencia de entrada del decoder (target)
        # Esta es la que se usará en combinación con la look_ahead_mask

        dec_target_padding_mask = tf.cast(tf.not_equal(decoder_inputs, 0), tf.float32)
        dec_target_padding_mask = dec_target_padding_mask[:, tf.newaxis, tf.newaxis, :] # Ajustamos las dimensiones

        # La look_ahead_mask y la dec_target_padding_mask se combinan para
        # el primer MHA (self-attention) en cada DecoderLayer.
        combined_mask = tf.maximum(look_ahead_mask, dec_target_padding_mask)

        dec_output = self.decoder(x_decoder, enc_output, training=training, look_ahead_mask=combined_mask, padding_mask=enc_mask)# encoder_padding_mask

        final_output = self.final_layer(dec_output) # Capa de prediccion de siguiente Token
        return final_output

## Dataset eng-fren

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("devicharith/language-translation-englishfrench")

print("Path to dataset files:", path)

100%|██████████| 3.51M/3.51M [00:00<00:00, 146MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/devicharith/language-translation-englishfrench/versions/2


In [ ]:
import os
import pandas as pd

file_path = os.path.join(path, 'eng_-french.csv')
df = pd.read_csv(file_path)

display(df.head())

,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


### Preparar el dataset
1. Inspeccionar el dataset
2. Preprocesar texto para limpiarlo y normalizarlo.
3. Agregar los tokens de inicio y fin al target (Frances)
4. Dividir los datos en train y validacion.
5. Construir el vocabulario compartido.
6. Tokenizar y numeralizar el texto.
7. Hacer el padding de las secuencias y crear los batches.
8. Verificar el paso 8.

Consideraciones: si es necesario, filtrar secuencias de mas de 100 tokens de longitud. Vamos a utilizar subword tokenization con Byte-Pair Encoding (BPE).

padding_ingles  = (seq_len_en, batch_size)
padding_frances  = (seq_len_fr, batch_size)

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import TextVectorization

In [ ]:
# 1. Cargar y preprocesar datos
def load_and_preprocess_data(file_path):

    # Cargar el dataset
    df = pd.read_csv(file_path)

    # Preprocesamiento de texto
    def preprocess_text(text):
        text = text.lower()
        text = re.sub(r"['\"()]", "", text)  # Eliminar comillas/paréntesis
        text = re.sub(r"[.!?]", r" \g<0> ", text)  # Añadir espacios alrededor de puntuación
        text = re.sub(r"\s+", " ", text).strip()  # Eliminar espacios extras
        return text

    # Aplicar preprocesamiento
    df['english'] = df['English words/sentences'].apply(preprocess_text)
    df['french'] = df['French words/sentences'].apply(preprocess_text)

    # Añadir tokens especiales al francés (target)
    df['french'] = df['french'].apply(lambda x: "[SOS] " + x + " [EOS]")

    return df[['english', 'french']]

In [ ]:
df = load_and_preprocess_data(file_path)

In [ ]:
df.tail()

,english,french
175616,"top-down economics never works, said obama . t...",[SOS] « léconomie en partant du haut vers le b...
175617,a carbon footprint is the amount of carbon dio...,[SOS] une empreinte carbone est la somme de po...
175618,death is something that were often discouraged...,[SOS] la mort est une chose quon nous décourag...
175619,since there are usually multiple websites on a...,[SOS] puisquil y a de multiples sites web sur ...
175620,if someone who doesnt know your background say...,[SOS] si quelquun qui ne connaît pas vos antéc...


In [ ]:
# 3. Dividir el dataset
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

print(f"Ejemplos entrenamiento: {len(train_df)}, Validación: {len(val_df)}")
print(train_df.head())

Ejemplos entrenamiento: 158058, Validación: 17563
                                                english  \
158383  they kept him waiting outside for a long time .   
146722       how much money did you spend on your car ?   
120085              i heard it from a reliable source .   
152460     my parents met each other in the mountains .   
63136                        my teacher drove me home .   

                                                   french  
158383      [SOS] ils le firent poireauter dehors . [EOS]  
146722  [SOS] combien dargent avez-vous dépensé pour v...  
120085    [SOS] je lai entendu dune source fiable . [EOS]  
152460  [SOS] mes parents se sont rencontrés dans les ...  
63136   [SOS] mon professeur ma reconduit chez moi . [...  


In [ ]:
# Calcular longitud máxima de secuencia en inglés y francés
max_seq_length_en_train = train_df['english'].apply(lambda x: len(x.split())).max()
max_seq_length_fr_train = train_df['french'].apply(lambda x: len(x.split())).max()

max_seq_length_en_val = val_df['english'].apply(lambda x: len(x.split())).max()
max_seq_length_fr_val = val_df['french'].apply(lambda x: len(x.split())).max()

print(f"Longitud máxima de secuencia en inglés (entrenamiento): {max_seq_length_en_train}")
print(f"Longitud máxima de secuencia en francés (entrenamiento): {max_seq_length_fr_train}")
print(f"Longitud máxima de secuencia en inglés (validación): {max_seq_length_en_val}")
print(f"Longitud máxima de secuencia en francés (validación): {max_seq_length_fr_val}")

Longitud máxima de secuencia en inglés (entrenamiento): 46
Longitud máxima de secuencia en francés (entrenamiento): 59
Longitud máxima de secuencia en inglés (validación): 33
Longitud máxima de secuencia en francés (validación): 43


### Creamos la instancia del modelo.

In [ ]:
d_model = 100 #512
num_layers = 1 #6
num_heads = 1 #8
dff = d_model*4
target_vocab_size = 31247 #37000
input_vocab_size = 15948
rate = 0.1

max_seq_length = 59
BATCH_SIZE = 128
#pe_input = max_seq_length
#pe_target = max_seq_length
transformer = Transformer(
        num_layers= num_layers,
        d_model= d_model,
        num_heads= num_heads,
        dff=dff,
        input_vocab_size=input_vocab_size,
        target_vocab_size=target_vocab_size,
        pe_input= max_seq_length,
        pe_target=max_seq_length)

In [ ]:
# 5. Adaptar los vectorizadores a los textos
vectorize_input = transformer.pre_layer_encoder.adapt(train_df["english"])
vectorize_target = transformer.pre_layer_decoder.adapt(train_df["french"])
#Asegurarse que ambos train y val tengan los mismos tokens.
# Obtener vocabulario
source_vocab = transformer.pre_layer_encoder.get_vocabulary()
target_vocab = transformer.pre_layer_decoder.get_vocabulary()

In [ ]:
print(source_vocab)
print(target_vocab)

['', '[UNK]', np.str_('.'), np.str_('i'), np.str_('you'), np.str_('to'), np.str_('the'), np.str_('?'), np.str_('a'), np.str_('is'), np.str_('tom'), np.str_('that'), np.str_('he'), np.str_('do'), np.str_('of'), np.str_('it'), np.str_('this'), np.str_('in'), np.str_('me'), np.str_('have'), np.str_('dont'), np.str_('was'), np.str_('my'), np.str_('for'), np.str_('are'), np.str_('im'), np.str_('your'), np.str_('what'), np.str_('we'), np.str_('be'), np.str_('want'), np.str_('not'), np.str_('she'), np.str_('know'), np.str_('like'), np.str_('on'), np.str_('with'), np.str_('can'), np.str_('were'), np.str_('his'), np.str_('youre'), np.str_('did'), np.str_('all'), np.str_('at'), np.str_('how'), np.str_('think'), np.str_('go'), np.str_('they'), np.str_('him'), np.str_('its'), np.str_('and'), np.str_('about'), np.str_('cant'), np.str_('time'), np.str_('here'), np.str_('very'), np.str_('didnt'), np.str_('get'), np.str_('there'), np.str_('no'), np.str_('as'), np.str_('will'), np.str_('her'), np.str_(

In [ ]:
print(len(source_vocab))
print(len(target_vocab))

15948
31247


In [ ]:
# 6. Mapeo de tokens especiales
# Encontrar índices de tokens especiales
sos_idx = target_vocab.index("[SOS]")
eos_idx = target_vocab.index("[EOS]")
empty_string_tensor = tf.constant([""])
pad_idx = vectorize_input(empty_string_tensor).numpy()[0, 0]  # Índice de padding

print(f"\nTokens especiales: SOS={sos_idx}, EOS={eos_idx}, PAD={pad_idx}")
print(f"Tamaño vocabulario inglés: {len(source_vocab)}, francés: {len(target_vocab)}")


Tokens especiales: SOS=2, EOS=3, PAD=0
Tamaño vocabulario inglés: 15948, francés: 31247


In [ ]:
# 7. Crear datasets de TensorFlow
def prepare_dataset(eng_batch, fr_batch):
    # Vectorizar batches
    encoder_inputs = vectorize_input(eng_batch)
    decoder_inputs = vectorize_target(fr_batch)

    # Crear entradas y etiquetas para el decoder
    # Entradas: [SOS] + oración
    # Etiquetas: oración + [EOS]
    labels = decoder_inputs[:, 1:]
    decoder_inputs = decoder_inputs[:, :-1]

    return (encoder_inputs, decoder_inputs), labels

# Crear datasets
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_df["english"], train_df["french"]))
val_ds = tf.data.Dataset.from_tensor_slices(
    (val_df["english"], val_df["french"]))

# Aplicar preparación y batching
train_ds = train_ds.batch(BATCH_SIZE).map(
    prepare_dataset, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).map(
    prepare_dataset, num_parallel_calls=tf.data.AUTOTUNE)

# Optimización de rendimiento
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

# 8. Verificar un batch
for (encoder_inputs, decoder_inputs), labels in train_ds.take(1):
    print("\nBatch de ejemplo:")
    print(f"Encoder inputs shape: {encoder_inputs.shape}")  # (batch, seq_len)
    print(f"Decoder inputs shape: {decoder_inputs.shape}")
    print(f"Labels shape: {labels.shape}")

    # Decodificar una muestra
    sample_idx = 0
    print("\nTexto original (inglés):", train_df["english"].iloc[sample_idx])
    print("Texto decodificado (inglés):",
          " ".join(source_vocab[idx] for idx in encoder_inputs[sample_idx].numpy() if idx != pad_idx))

    print("\nTexto original (francés):", train_df["french"].iloc[sample_idx])
    print("Decoder input (francés):",
          " ".join(target_vocab[idx] for idx in decoder_inputs[sample_idx].numpy() if idx != pad_idx))
    print("Labels (francés):",
          " ".join(target_vocab[idx] for idx in labels[sample_idx].numpy() if idx != pad_idx))


Batch de ejemplo:
Encoder inputs shape: (128, 59)
Decoder inputs shape: (128, 59)
Labels shape: (128, 59)

Texto original (inglés): they kept him waiting outside for a long time .
Texto decodificado (inglés): they kept him waiting outside for a long time .

Texto original (francés): [SOS] ils le firent poireauter dehors . [EOS]
Decoder input (francés): [SOS] ils le firent poireauter dehors . [EOS]
Labels (francés): ils le firent poireauter dehors . [EOS]


#Entrenamiento

In [ ]:
import math

# Parámetros del usuario
num_examples    = len(train_df)
num_epochs      = 100
warmup_ratio    = 0.10

# Cálculo de pasos
steps_per_epoch = math.ceil(num_examples / BATCH_SIZE)
total_steps     = steps_per_epoch * num_epochs
warmup_steps    = int(total_steps * warmup_ratio)

print(f"Total steps: {total_steps}, Warm‑up steps (@{warmup_ratio*100:.0f}%): {warmup_steps}")

class WarmUpThenDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, base_lr, warmup_steps, total_steps):
        super().__init__()
        self.base_lr = base_lr
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps

    def __call__(self, step):
        # warm-up lineal
        if step < self.warmup_steps:
            return self.base_lr * (step / tf.cast(self.warmup_steps, tf.float32))
        # decay inverso de raíz cuadrada
        return self.base_lr * tf.math.rsqrt(tf.cast(step, tf.float32))

base_lr     = 1e-4
lr_schedule = WarmUpThenDecay(base_lr, warmup_steps, total_steps)
optimizer   = tf.keras.optimizers.Adam(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.98,
    epsilon=1e-9
)

# Compilado
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

transformer.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

Total steps: 123500, Warm‑up steps (@10%): 12350


In [ ]:
# Verificar conexión de gradientes
sample_batch = next(iter(train_ds.take(1)))

with tf.GradientTape() as tape:
    outputs = transformer(sample_batch[0], training=True)
    loss = loss_function(sample_batch[1], outputs)

grads = tape.gradient(loss, transformer.trainable_variables)
missing_grads = [v.name for v, g in zip(transformer.trainable_variables, grads) if g is None]

if missing_grads:
    print("¡Problema! Gradientes faltantes para:")
    for name in missing_grads:
        print(f" - {name}")
else:
    print("¡Todo correcto! Todos los gradientes están presentes")

¡Todo correcto! Todos los gradientes están presentes


In [ ]:
history = transformer.fit(
    train_ds,
    epochs=num_epochs,
    validation_data=val_ds
)

Epoch 1/100
 872/1235 ━━━━━━━━━━━━━━━━━━━━ 1:28:38 15s/step - accuracy: 0.0376 - loss: 5.5032

In [ ]:
transformer.summary()